In [3]:
!pip install pandas

!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------- ------------------------ 4.5/11.5 MB 22.3 MB/s eta 0:00:01
   --------------------------------- ------ 9.7/11.5 MB 26.2 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 22.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------------- ------------------------ 5.0/12.6 MB 25.2 MB/s eta 0:00:01
   ------------------------------- -------- 10.0/12.6 MB 24.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 22.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np

In [21]:
def load_data(play_tennis):
    data = pd.read_csv('play_tennis.csv')
    return data


In [22]:
df.head(15) 

day   outlook  temp humidity    wind play
0    D1     Sunny   Hot     High    Weak   No
1    D2     Sunny   Hot     High  Strong   No
2    D3  Overcast   Hot     High    Weak  Yes
3    D4      Rain  Mild     High    Weak  Yes
4    D5      Rain  Cool   Normal    Weak  Yes
5    D6      Rain  Cool   Normal  Strong   No
6    D7  Overcast  Cool   Normal  Strong  Yes
7    D8     Sunny  Mild     High    Weak   No
8    D9     Sunny  Cool   Normal    Weak  Yes
9   D10      Rain  Mild   Normal    Weak  Yes
10  D11     Sunny  Mild   Normal  Strong  Yes
11  D12  Overcast  Mild     High  Strong  Yes
12  D13  Overcast   Hot   Normal    Weak  Yes
13  D14      Rain  Mild     High  Strong   No

In [40]:


def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy_value = -np.sum((counts / np.sum(counts)) * np.log2(counts / np.sum(counts)))
    return entropy_value

def information_gain(data, split_attribute, target_attribute):
    total_entropy = entropy(data[target_attribute])
    print(f"Total Entropy of target ({target_attribute}): {total_entropy}")
    values, counts = np.unique(data[split_attribute], return_counts=True)
    weighted_entropy = np.sum(
        [(counts[i] / np.sum(counts)) * entropy(data[data[split_attribute] == values[i]][target_attribute])
         for i in range(len(values))]
    )
    print(f"Weighted Entropy for {split_attribute}: {weighted_entropy}")
    info_gain = total_entropy - weighted_entropy
    print(f"Information Gain for {split_attribute}: {info_gain}")
    return info_gain

if __name__ == "__main__":
    file_path = "/mnt/data/play_tennis.csv"
    data = load_data(file_path)
    target_attribute = "play"
    features = [col for col in data.columns if col not in ["day", target_attribute]]

    print("Calculating Entropy and Information Gain:")
    info_gains = {}
    for feature in features:
        info_gains[feature] = information_gain(data, feature, target_attribute)

    root_node = max(info_gains, key=info_gains.get)
    print(f"Root Node: {root_node}")



Calculating Entropy and Information Gain:
Total Entropy of target (play): 0.9402859586706311
Weighted Entropy for outlook: 0.6935361388961918
Information Gain for outlook: 0.24674981977443933
Total Entropy of target (play): 0.9402859586706311
Weighted Entropy for temp: 0.9110633930116763
Information Gain for temp: 0.02922256565895487
Total Entropy of target (play): 0.9402859586706311
Weighted Entropy for humidity: 0.7884504573082896
Information Gain for humidity: 0.15183550136234159
Total Entropy of target (play): 0.9402859586706311
Weighted Entropy for wind: 0.8921589282623617
Information Gain for wind: 0.04812703040826949
Root Node: outlook


Info Gain:


In [ ]:
# Build the decision tree
def build_tree(data, original_data, features, target_attribute, parent_node_class=None):
    # Stopping criteria
    # If all target values have the same value, return that value
    if len(np.unique(data[target_attribute])) <= 1:
        return np.unique(data[target_attribute])[0]
    
    # If the dataset is empty, return the mode of the original dataset's target feature
    elif len(data) == 0:
        return np.unique(original_data[target_attribute])[
            np.argmax(np.unique(original_data[target_attribute], return_counts=True)[1])
        ]

    # If the feature space is empty, return the parent node class
    elif len(features) == 0:
        return parent_node_class

    # Otherwise, build the tree recursively
    else:
        # Parent node class: mode target feature value of the current dataset
        parent_node_class = np.unique(data[target_attribute])[
            np.argmax(np.unique(data[target_attribute], return_counts=True)[1])
        ]

        # Select the feature with the highest information gain
        item_values = [information_gain(data, feature, target_attribute) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        # Create the tree structure
        tree = {best_feature: {}}

        # Remove the best feature from the feature list
        features = [f for f in features if f != best_feature]

        # Grow a branch under the root node for each possible value of the feature
        for value in np.unique(data[best_feature]):
            subset = data[data[best_feature] == value]
            subtree = build_tree(subset, data, features, target_attribute, parent_node_class)
            tree[best_feature][value] = subtree

        return tree

# Print the decision tree
def print_tree(tree, indent=""):
    if not isinstance(tree, dict):
        print(indent + str(tree))
    else:
        for key, value in tree.items():
            print(indent + str(key))
            for subkey, subvalue in value.items():
                print(indent + "|-- " + str(subkey))
                print_tree(subvalue, indent + "|   ")

# Main program
if __name__ == "__main__":
    # Load the dataset
    file_path = "/mnt/data/play_tennis.csv"
    data = load_data(file_path)

    # Define the target and features
    target_attribute = "play"
    features = [col for col in data.columns if col not in ["day", target_attribute]]

    # Build the decision tree
    decision_tree = build_tree(data, data, features, target_attribute)

    # Print the decision tree
    print("Decision Tree:")
    print_tree(decision_tree)
